### API Router



### Prompt Template

User input does not need to be passed directly into the LLM. The prompt template provides additional context on the specific task at hand.

Prompt Templates are designed for single-turn interactions. It typically accepts a single template string with placeholders that can be dynamically filled with inputs.

```python
from langchain.prompts import PromptTemplate

template = "Translate the following English text to French: {text}"
prompt = PromptTemplate(input_variables=["text"], template=template)

filled_prompt = prompt.format(text="Hello, how are you?")
print(filled_prompt)
# Output: Translate the following English text to French: Hello, how are you?
```

### Chat Prompt Template

ChatPromptTemplate is used for multi-turn interactions or prompts designed for chat-based language models. It is composed of messages like SystemMessage, HumanMessage, and AIMessage. These represent the context, user input, and AI responses, respectively.

ChatPromptTemplate can take messages in various formats, including predefined message classes (e.g., SystemMessage, HumanMessage) as well as tuple formats. LangChain provides message classes such as SystemMessage, HumanMessage, AIMessage, and ChatMessage. These are structured representations for different roles in a chat.

```python
from langchain.prompts import ChatPromptTemplate, SystemMessage, HumanMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Can you help me with my math homework?")
])
formatted_messages = chat_prompt.format_messages()
for msg in formatted_messages:
    print(f"{msg.type}: {msg.content}")
```

Instead of using the predefined message classes, you can represent messages as tuples, where the first element is the role (e.g., "system", "human", "assistant"), and the second element is the content of the message.

```python
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Can you help me solve a quadratic equation?"),
    ("assistant", "Of course! What's the equation?")
])
formatted_messages = chat_prompt.format_messages()
for msg in formatted_messages:
    print(f"{msg.type}: {msg.content}")
```

### Chat Prompt Template and Message Placeholder

The MessagesPlaceholder in LangChain is a utility used within ChatPromptTemplate to insert dynamic chat histories or message sequences into a prompt. This is particularly useful when you want to incorporate conversation history dynamically without hardcoding all previous messages into the prompt.

MessagesPlaceholder is commonly used when:
- Maintaining Context: You want to include past messages in a chat.
- Dynamic History: The history may change or be trimmed based on application logic.
- Trimming: You can preprocess the chat_history (e.g., trim irrelevant messages) before passing it to the prompt.

Here’s how to use MessagesPlaceholder in a ChatPromptTemplate:

```python
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("What do you think about {topic}?")
])
```

At runtime, you pass the actual chat history to replace the MessagesPlaceholder:

```python
from langchain.schema import AIMessage, HumanMessage

chat_history = [
    HumanMessage(content="Can you summarize quantum physics?"),
    AIMessage(content="Sure! Quantum physics deals with subatomic particles and wave-particle duality."),
]

formatted_messages = chat_prompt.format_messages(
    chat_history=chat_history,  # Replace placeholder
    topic="black holes"
)

for message in formatted_messages:
    print(f"{message.type}: {message.content}")
```

### Chat Prompt Template and LCEL

PromptTemplate and ChatPromptTemplate implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

PromptTemplate accepts a dictionary (of the prompt variables) and returns a StringPromptValue. A ChatPromptTemplate accepts a dictionary and returns a ChatPromptValue.

```python
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val # StringPromptValue(text='Tell me a funny joke about chickens.')
prompt_val.to_string() # 'Tell me a funny joke about chickens.'
prompt_val.to_messages() # [HumanMessage(content='Tell me a funny joke about chickens.')]

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val.to_messages()
# [SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
```

### Few Shot Prompt Templates and Example Selectors

The purpose of Example Selectors in LangChain is to dynamically choose the most relevant examples to include in a prompt. This is especially important when you have a large dataset of examples and cannot include all of them due to constraints like token limits or relevance to the current input. Example selectors ensure that the chosen examples are tailored to the specific input, improving the quality of responses from the language model.

**Large language models like GPT often perform better with few-shot learning, where the prompt includes input-output examples to demonstrate the task. Including irrelevant or too many examples can lead to suboptimal responses or exceed token limits.**

**Example Selectors dynamically choose the most relevant subset of examples based on the current input, ensuring optimal context is provided to the model.**

LangChain provides several prebuilt example selectors, and you can also implement custom ones. Common types include:
- Similarity Selector
    - Uses semantic similarity between inputs and examples to decide which examples to choose.
- Max Marginal Relevance (MMR) Selector
    - Balances relevance and diversity by selecting examples that are similar to the input but not too similar to each other.
- Length-Based Selector
    - Chooses examples that fit within a token limit or are of a similar length to the input.
- N-gram Overlap Selector
    - Selects examples based on shared n-grams between the input and examples.

Select by length example selector selects which examples to use based on length. This is useful when you are worried about constructing a prompt that will go over the length of the context window. For longer inputs, it will select fewer examples to include, while for shorter inputs it will select more.

```python
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
```

The MaxMarginalRelevanceExampleSelector selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for diversity. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

```python
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)

mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# Input is a feeling, so should select the happy/sad example as the first one
print(mmr_prompt.format(adjective="worried"))
Give the antonym of every input

Input: happy
Output: sad

Input: windy
Output: calm

Input: worried
Output:


# Let's compare this to what we would just get if we went solely off of similarity,
# by using SemanticSimilarityExampleSelector instead of MaxMarginalRelevanceExampleSelector.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))
Give the antonym of every input

Input: happy
Output: sad

Input: sunny
Output: gloomy

Input: worried
Output:
```

The NGramOverlapExampleSelector selects and orders examples based on which examples are most similar to the input, according to an ngram overlap score. The ngram overlap score is a float between 0.0 and 1.0, inclusive.

The selector allows for a threshold score to be set. Examples with an ngram overlap score less than or equal to the threshold are excluded. The threshold is set to -1.0, by default, so will not exclude any examples, only reorder them. Setting the threshold to 0.0 will exclude examples that have no ngram overlaps with the input.

```python
from langchain_community.example_selector.ngram_overlap import (
    NGramOverlapExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a fictional translation task.
examples = [
    {"input": "See Spot run.", "output": "Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."},
]

example_selector = NGramOverlapExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The threshold, at which selector stops.
    # It is set to -1.0 by default.
    threshold=-1.0,
    # For negative threshold:
    # Selector sorts examples by ngram overlap score, and excludes none.
    # For threshold greater than 1.0:
    # Selector excludes all examples, and returns an empty list.
    # For threshold equal to 0.0:
    # Selector sorts examples by ngram overlap score,
    # and excludes those with no ngram overlap with input.
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the Spanish translation of every input",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)

# An example input with large ngram overlap with "Spot can run."
# and no overlap with "My dog barks."
print(dynamic_prompt.format(sentence="Spot can run fast."))
Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: My dog barks.
Output: Mi perro ladra.

Input: Spot can run fast.
Output:

Select by similarity selects examples based on similarity to the inputs. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs.

```python
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # The number of examples to produce.
    k=1,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))
Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:
```

### Few Shot Chat Message Prompt Template

FewShotPromptTemplate is a general-purpose template for few-shot prompting. Examples are stored as plain text templates or dictionaries. Examples are formatted using a simple PromptTemplate. Includes a prefix (instructions or context) and a suffix (the current input or question). Dynamically selects the most relevant examples using an ExampleSelector (e.g., SemanticSimilarityExampleSelector).

FewShotChatMessagePromptTemplate is specifically designed for chat-based models (e.g., ChatGPT, Anthropic, or any model that uses chat messages). Formats examples into chat-style messages, distinguishing between system, human, and ai roles. Each example is converted into a sequence of chat messages (e.g., Human: Input\nAI: Output). Uses ChatPromptTemplate to define message roles. Can dynamically insert examples in a conversational format using ExampleSelector. Suitable for conversational tasks where examples need to be formatted as chat exchanges.

```python
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "3+3", "output": "6"},
]

few_shot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_chat_prompt.format())
Human: 2+2
AI: 4
Human: 3+3
AI: 6

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_chat_prompt,
        ("human", "{input}"),
    ]
)

from langchain_community.chat_models import ChatAnthropic

chain = final_prompt | ChatAnthropic(temperature=0.0)

chain.invoke({"input": "What's the square of a triangle?"})
AIMessage(content=' Triangles do not have a "square". A square refers to a shape with 4 equal sides and 4 right angles. Triangles have 3 sides and 3 angles.\n\nThe area of a triangle can be calculated using the formula:\n\nA = 1/2 * b * h\n\nWhere:\n\nA is the area \nb is the base (the length of one of the sides)\nh is the height (the length from the base to the opposite vertex)\n\nSo the area depends on the specific dimensions of the triangle. There is no single "square of a triangle". The area can vary greatly depending on the base and height measurements.', additional_kwargs={}, example=False)
```

Sometimes you may want to condition which examples are shown based on the input. For this, you can replace the examples with an example_selector.

```python
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

from langchain_community.chat_models import ChatAnthropic

chain = final_prompt | ChatAnthropic(temperature=0.0)

chain.invoke({"input": "What's 3+3?"})
# AIMessage(content=' 3 + 3 = 6', additional_kwargs={}, example=False)
```

### Partial Prompt Templates

It can make sense to "partial" a prompt template - e.g. pass in a subset of the required values, as to create a new prompt template which expects only the remaining subset of values.

One common use case for wanting to partial a prompt template is if you get some of the variables before others. For example, suppose you have a prompt template that requires two variables, foo and baz. If you get the foo value early on in the chain, but the baz value later, it can be annoying to wait until you have both variables in the same place to pass them to the prompt template. Instead, you can partial the prompt template with the foo value, and then pass the partialed prompt template along and just use that. Below is an example of doing this:

```python
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))
```

The other common use is to partial with a function. The use case for this is when you have a variable you know that you always want to fetch in a common way. A prime example of this is with date or time. Imagine you have a prompt which you always want to have the current date. You can't hard code it in the prompt, and passing it along with the other input variables is a bit annoying. In this case, it's very handy to be able to partial the prompt with a function that always returns the current date.

```python
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))
```

You can also just initialize the prompt with the partialed variables, which often makes more sense in this workflow.

```python
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))
```

### Prompt Composition

LangChain provides a user friendly interface for composing different parts of prompts together. You can do this with either string prompts or chat prompts. Constructing prompts this way allows for easy reuse.

When working with string prompts, each template is joined together. You can work with either prompts directly or strings:

```python
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
prompt
# PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')
```

LangChain includes an abstraction PipelinePromptTemplate, which can be useful when you want to reuse parts of prompts. 

```python
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables # ['example_q', 'person', 'input', 'example_a']

print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)
You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:
```


### Chat Model

Chat Models are built on top of LLMs. The LLM objects take string as input and output string. The ChatModel objects take a list of messages as input and output a message. While chat models use language models under the hood, the interface they use is a bit different. Rather than using a "text in, text out" API, they use an interface where "chat messages" are the inputs and outputs.

**A chat model is a language model that uses chat messages as inputs and returns chat messages as outputs (as opposed to using plain text).** LangChain has integrations with many model providers (OpenAI, Cohere, Hugging Face, etc.) and exposes a standard interface to interact with all of these models.

The chat model interface is based around messages rather than raw text. The types of messages currently supported in LangChain are AIMessage, HumanMessage, SystemMessage, FunctionMessage and ChatMessage -- ChatMessage takes in an arbitrary role parameter. Most of the time, you'll just be dealing with HumanMessage, AIMessage, and SystemMessage.

### Chat Model and LCEL

Chat models implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

Chat models accept List[BaseMessage] as inputs, or objects which can be coerced to messages, including str (converted to HumanMessage) and PromptValue.

```python
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]

# using invoke
chat.invoke(messages)
# AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to fit the noise in the training data, leading to poor generalization on unseen data. Regularization techniques introduce additional constraints or penalties to the model's objective function, discouraging it from becoming overly complex and promoting simpler and more generalizable models. Regularization helps to strike a balance between fitting the training data well and avoiding overfitting, leading to better performance on new, unseen data.")

# using stream
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)
# The purpose of model regularization is to prevent overfitting and improve the generalization of a machine learning model. Overfitting occurs when a model is too complex and learns the noise or random variations in the training data, which leads to poor performance on new, unseen data. Regularization techniques introduce additional constraints or penalties to the model's learning process, discouraging it from fitting the noise and reducing the complexity of the model. This helps to improve the model's ability to generalize well and make accurate predictions on unseen data.    

# using batch
chat.batch([messages])
# [AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to learn the noise or random fluctuations in the training data, rather than the underlying patterns or relationships. Regularization techniques add a penalty term to the model's objective function, which discourages the model from becoming too complex and helps it generalize better to new, unseen data. This improves the model's ability to make accurate predictions on new data by reducing the variance and increasing the model's overall performance.")]

# using ainvoke
await chat.ainvoke(messages)
# AIMessage(content='The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to memorize the training data instead of learning general patterns and relationships. This leads to poor performance on new, unseen data.\n\nRegularization techniques introduce additional constraints or penalties to the model during training, discouraging it from becoming overly complex. This helps to strike a balance between fitting the training data well and generalizing to new data. Regularization techniques can include adding a penalty term to the loss function, such as L1 or L2 regularization, or using techniques like dropout or early stopping. By regularizing the model, it encourages it to learn the most relevant features and reduce the impact of noise or outliers in the data.')

# using astream
async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)
# The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to memorize the training data instead of learning the underlying patterns. Regularization techniques help in reducing the complexity of the model by adding a penalty to the loss function. This penalty encourages the model to have smaller weights or fewer features, making it more generalized and less prone to overfitting. The goal is to find the right balance between fitting the training data well and being able to generalize well to unseen data.

# using astream log
async for chunk in chat.astream_log(messages):
    print(chunk) 
```

### Chat Models and Langsmith

All ChatModels come with built-in LangSmith tracing. Just set the following environment variables:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY=<your-api-key>
```

and any ChatModel invocation (whether it's nested in a chain or not) will automatically be traced. A trace will include inputs, outputs, latency, token usage, invocation params, environment params, and more.

In LangSmith you can then provide feedback for any trace, compile annotated datasets for evals, debug performance in the playground, and more.

### Chat Model Message types

ChatModels take a list of messages as input and return a message. There are a few different types of messages. All messages have a role and a content property. The role describes WHO is saying the message. LangChain has different message classes for different roles. The content property describes the content of the message. This can be a few different things:
- A string (most models deal this type of content)
- A List of dictionaries (this is used for multi-modal input, where the dictionary contains information about that input type and that input location)

In addition, messages have an additional_kwargs property. This is where additional information about messages can be passed. This is largely used for input parameters that are provider specific and not general. The best known example of this is function_call from OpenAI.

Message Types:
- HumanMessage
    - This represents a message from the user. Generally consists only of content.
- AIMessage
    - This represents a message from the model. This may have additional_kwargs in it - for example tool_calls if using OpenAI tool calling.
- SystemMessage
    - This represents a system message, which tells the model how to behave. This generally only consists of content. Not every model supports this.
- FunctionMessage
    - This represents the result of a function call. In addition to role and content, this message has a name parameter which conveys the name of the function that was called to produce this result.
- ToolMessage
    - This represents the result of a tool call. This is distinct from a FunctionMessage in order to match OpenAI's function and tool message types. In addition to role and content, this message has a tool_call_id parameter which conveys the id of the call to the tool that was called to produce this result.

### Chat Models and Streaming

All ChatModels implement the Runnable interface, which comes with default implementations of all methods, ie. ainvoke, batch, abatch, stream, astream. This gives all ChatModels basic support for streaming.

**Streaming support defaults to returning an Iterator (or AsyncIterator in the case of async streaming)** of a single value, the final result returned by the underlying ChatModel provider. This obviously doesn't give you token-by-token streaming, which requires native support from the ChatModel provider, but ensures your code that expects an iterator of tokens can work for any of our ChatModel integrations.

### Chat Models and Tool calling

Tool Calling in LangChain is a feature that allows a language model to generate structured output by simulating the "calling" of a predefined tool. This process involves the model creating arguments that match a user-defined schema for a tool, but the actual execution of the tool (or even whether the tool is executed) is left to the user.

When we define a tool in LangChain, we specify:
- What the tool does: Its purpose or functionality.
- What inputs (arguments) it requires: A schema that defines the parameters the tool needs to execute.

The model "creates arguments" by generating these input values based on the user's query or prompt. Consider a tool for basic arithmetic operations:

```python
class CalculatorTool(BaseTool):
    name = "calculate"
    args_schema = {
        "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
        "num1": {"type": "number"},
        "num2": {"type": "number"},
    }
```
- Operation: Specifies the type of calculation (e.g., "add").
- The first number for the operation (e.g., 5).
- The second number for the operation (e.g., 7).

When a user provides input like: "Add 5 and 7." The model interprets the query and generates the following arguments:

```python
{
    "operation": "add",
    "num1": 5,
    "num2": 7
}
```
These arguments match the schema defined in the example CalculatorTool.

In effect, despite the name, tool calling does not involve the model directly performing actions. Instead, the model provides parameters or arguments that conform to the schema, which can then be used by the user to trigger the tool or extract structured data.

How Tool Calling Works:
- Define the Tool:
    - A "tool" in LangChain represents an action the model can "call" by generating arguments that match a predefined schema.
    - The schema specifies the inputs the tool accepts and can include validations or constraints.
- Prompt the Model:
    - The user provides a prompt or query to the model.
    - The model generates output that matches the schema of the tool, simulating a "call" to the tool.
- User Handles the Execution:
    - The user decides whether to actually execute the tool based on the generated arguments.
    - Alternatively, the user may treat the structured output directly as the final result.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools. For instance, given a search engine tool, an LLM might handle a query by first issuing a call to the search engine. The system calling the LLM can receive the tool call, execute it, and return the output to the LLM to inform its response. LangChain includes a suite of built-in tools and supports several methods for defining your own custom tools. Tool-calling is extremely useful for building tool-using chains and agents, and for getting structured outputs from models more generally.

Providers adopt different conventions for formatting tool schemas and tool calls. For instance, Anthropic returns tool calls as parsed structures within a larger content block:
```json
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```

whereas OpenAI separates tool calls into a distinct parameter, with arguments as JSON strings:
```json
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```

LangChain implements standard interfaces for defining tools, passing them to LLMs, and representing tool calls.

**For a model to be able to invoke tools, you need to pass tool schemas to it when making a chat request. LangChain ChatModels supporting tool calling features implement a .bind_tools method, which receives a list of LangChain tool objects, Pydantic classes, or JSON Schemas and binds them to the chat model in the provider-specific expected format.** That's a mouthful. Let's dissect this:
- A tool in LangChain represents a predefined action the model can "invoke" by generating structured arguments that match a schema.
- Binding Tools to the Model
    - Tools must be bound to the model so that the model knows which tools are available and can generate arguments for them.
    - Binding tools involves passing the tool schemas to the model, which adapts them to the specific format expected by the provider’s API (e.g., OpenAI, Anthropic).
- .bind_tools() Method
    - The .bind_tools() method in LangChain allows you to attach tools to a ChatModel instance.
    - Once tools are bound, every subsequent API call to the model will include the tool schemas, enabling the model to simulate invoking them.

Example:

```python
from langchain.tools import BaseTool

class CalculatorTool(BaseTool):
    name = "calculate"
    description = "Performs arithmetic operations"
    args_schema = {
        "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
        "num1": {"type": "number"},
        "num2": {"type": "number"},
    }

    def _run(self, operation, num1, num2):
        if operation == "add":
            return num1 + num2
        elif operation == "subtract":
            return num1 - num2
        elif operation == "multiply":
            return num1 * num2
        elif operation == "divide":
            return num1 / num2
        else:
            raise ValueError("Invalid operation")

# Use .bind_tools() to attach the tools to the model.
from langchain.chat_models import ChatOpenAI

# Instantiate the model
llm = ChatOpenAI(temperature=0)

# Instantiate the tool
calculator_tool = CalculatorTool()

# Bind the tool to the model
llm_with_tools = llm.bind_tools([calculator_tool])
```

How Binding Works:
- Schema Passing:
    - When you call .bind_tools(), LangChain converts the tool definitions into the specific JSON schema format expected by the provider (e.g., OpenAI or Anthropic).
- Model Awareness:
    - After binding, the model becomes "aware" of the tools and their schemas. Every subsequent chat request automatically includes the tool schemas as part of the API call.
- Tool Invocation by the Model:
    - During a conversation, the model can decide whether to "call" a tool by generating arguments that match the tool schema.

**When you interact with the bound model, the model can now include tool invocations in its responses.**

```python
response = llm_with_tools.invoke({
    "input": "Can you add 7 and 3?"
})

print(response)

# The model generates an output in the format:
{
    "tool": "calculate",
    "arguments": {
        "operation": "add",
        "num1": 7,
        "num2": 3
    }
}
```

We can define the schema for custom tools using the @tool decorator on Python functions:
```python
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
```

We can equivalently define the schema using Pydantic. Pydantic is useful when your tool inputs are more complex:

```python
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

# We can bind them to chat models as follows:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# We can use the bind_tools() method to handle converting Multiply to a "tool" and binding it to the model (i.e., passing it in each time the model is invoked).
llm_with_tools = llm.bind_tools(tools)
```

**When you just use bind_tools(tools), the model can choose whether to return one tool call, multiple tool calls, or no tool calls at all. Some models support a tool_choice parameter that gives you some ability to force the model to call a tool. For models that support this, you can pass in the name of the tool you want the model to always call tool_choice="xyz_tool_name". Or you can pass in tool_choice="any" to force the model to call at least one tool, without specifying which tool specifically.**

Currently tool_choice="any" functionality is supported by OpenAI, MistralAI, FireworksAI, and Groq. Currently Anthropic does not support tool_choice at all.

If we wanted our model to always call the multiply tool we could do:
```python
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")
```

And if we wanted it to always call at least one of add or multiply, we could do:
```python
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")
```

**If tool calls are included in a LLM response, they are attached to the corresponding AIMessage or AIMessageChunk (when streaming) as a list of ToolCall objects in the .tool_calls attribute.** A ToolCall is a typed dict that includes a tool name, dict of argument values, and (optionally) an identifier. Messages with no tool calls default to an empty list for this attribute.

Example:
```python
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls
# [{'name': 'multiply',
#   'args': {'a': 3, 'b': 12},
#   'id': 'call_UL7E2232GfDHIQGOM4gJfEDD'},
#  {'name': 'add',
#   'args': {'a': 11, 'b': 49},
#   'id': 'call_VKw8t5tpAuzvbHgdAXe9mjUx'}]
```

The .tool_calls attribute should contain valid tool calls. Note that on occasion, model providers may output malformed tool calls (e.g., arguments that are not valid JSON). When parsing fails in these cases, instances of InvalidToolCall are populated in the .invalid_tool_calls attribute. An InvalidToolCall can have a name, string arguments, identifier, and error message.

If desired, output parsers can further process the output. For example, we can convert back to the original Pydantic class:
```python
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)
# [multiply(a=3, b=12), add(a=11, b=49)]
```
How cool is that! We get back a pydantic object of the response from the model.

Streaming

**When tools are called in a streaming context, message chunks will be populated with tool call chunk objects in a list via the .tool_call_chunks attribute.** A ToolCallChunk includes optional string fields for the tool name, args, and id, and includes an optional integer field index that can be used to join chunks together. Fields are optional because portions of a tool call may be streamed across different chunks (e.g., a chunk that includes a substring of the arguments may have null values for the tool name and id).

```python
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)
# []
# [{'name': 'multiply', 'args': '', 'id': 'call_5Gdgx3R2z97qIycWKixgD2OU', 'index': 0}]
# [{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
# [{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
# [{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
# [{'name': None, 'args': '2}', 'id': None, 'index': 0}]
# [{'name': 'add', 'args': '', 'id': 'call_DpeKaF8pUCmLP0tkinhdmBgD', 'index': 1}]
# [{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
# [{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
# [{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
# [{'name': None, 'args': '49}', 'id': None, 'index': 1}]
# []
```

Note that adding message chunks will merge their corresponding tool call chunks. This is the principle by which LangChain's various tool output parsers support streaming. For example, below we accumulate tool call chunks:

```python
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)
# []
# [{'name': 'multiply', 'args': '', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a"', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, ', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a"', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11,', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11, "b": ', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11, "b": 49}', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11, "b": 49}', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
```

**If we're using the model-generated tool invocations to actually call tools and want to pass the tool results back to the model, we can do so using ToolMessages.** Let's explore this in-depth because it is so important:

```python
from langchain_core.messages import HumanMessage, ToolMessage

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages
# AIMessage(content='3 * 12 = 36\n11 + 49 = 60', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 209, 'total_tokens': 225}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-a55f8cb5-6d6d-4835-9c6b-7de36b2590c7-0')
```
- Tools (@tool): These are functions registered as tools that the model can "call."
- Each tool has a name, arguments, and a return type.
    - Two tools, add and multiply, are defined with argument types (int) and return types (int).
    - Example: add(a, b) adds two numbers.
    - Example: multiply(a, b) multiplies two numbers.
- The language model (LLM) is bound to the tools using bind_tools(tools).
    - The tools are "registered" with the LLM so that the model knows about them and can invoke them when appropriate.
    - This allows the model to "invoke" tools by generating tool calls.
- Initialize Messages: messages = [HumanMessage(query)]. A conversation starts with a HumanMessage containing the user’s query.
- The user sends a query (HumanMessage). The model responds (AIMessage) with tool invocations.
    - The LLM processes the input messages and generates an AIMessage response.
    - This response (ai_msg) contains tool calls in its tool_calls attribute.

For example, if the query is: "What is 3 multiplied by 12, and then add 11 to the result?" The AIMessage might include tool calls:
```json
[
    {"name": "multiply", "args": {"a": 3, "b": 12}, "id": "tool_call_1"},
    {"name": "add", "args": {"a": 11, "b": 49}, "id": "tool_call_2"}
]
```

The tools are executed, and their results (ToolMessages) are passed back to the model:
```python
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
```
- Iterate through the tool_calls in the AIMessage.
- Find the corresponding tool (add or multiply) based on the name in the tool call.
- Execute the tool with the provided args.
- After executing the tool, the result is added as a ToolMessage to the conversation.
- The ToolMessage includes:
    - The tool output (e.g., 36 or 60).
    - The tool_call_id linking the result to the specific tool invocation.

After processing the tool calls, the messages list contains:
- HumanMessage: The user’s query.
- AIMessage: The LLM’s response with tool calls.
- ToolMessages: The results of the tool executions.
```python
[
    HumanMessage(content="What is 3 multiplied by 12, and then add 11 to the result?"),
    AIMessage(
        content="3 * 12 = 36\n11 + 49 = 60",
        tool_calls=[
            {"name": "multiply", "args": {"a": 3, "b": 12}, "id": "tool_call_1"},
            {"name": "add", "args": {"a": 11, "b": 49}, "id": "tool_call_2"}
        ],
    ),
    ToolMessage(content=36, tool_call_id="tool_call_1"),
    ToolMessage(content=60, tool_call_id="tool_call_2")
]
```

Why Use ToolMessages?
- ToolMessages allow tool outputs to be integrated back into the conversation, enabling the model to continue reasoning with updated information.
- By including the tool_call_id, each ToolMessage explicitly links the result to its corresponding tool invocation, maintaining traceability.
- This setup enables dynamic and interactive workflows where tools and the model collaborate iteratively.
- The model can issue multiple tool calls, receive results, and use them to generate more informed responses.

It's very critical to understand the topic above!

Tools and Few-Shot Examples

For more complex tool use it's very useful to add few-shot examples to the prompt. We can do this by adding AIMessages with ToolCalls and corresponding ToolMessages to our prompt.

```python
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls
# [{'name': 'multiply',
#   'args': {'a': 119, 'b': 8},
#   'id': 'call_tWwpzWqqc8dQtN13CyKZCVMe'}]
```

### Chat Models and Structured Output

It is often crucial to have LLMs return structured output. This is because oftentimes the outputs of the LLMs are used in downstream applications, where specific arguments are required. Having the LLM return structured output reliably is necessary for that.

There are a few different high level strategies that are used to do this:
- Prompting: This is when you ask the LLM (very nicely) to return output in the desired format (JSON, XML). This is nice because it works with all LLMs. It is not nice because there is no guarantee that the LLM returns the output in the right format.
- Function calling: This is when the LLM is fine-tuned to be able to not just generate a completion, but also generate a function call. The functions the LLM can call are generally passed as extra parameters to the model API. The function names and descriptions should be treated as part of the prompt (they usually count against token counts, and are used by the LLM to decide what to do).
- Tool calling: A technique similar to function calling, but it allows the LLM to call multiple functions at the same time.
- JSON mode: This is when the LLM is guaranteed to return JSON.

Different models may support different variants of these, with slightly different parameters. In order to make it easy to get LLMs to return structured output, we have added a common interface to LangChain models: .with_structured_output.

By invoking this method (and passing in a JSON schema or a Pydantic model) the model will add whatever model parameters + output parsers are necessary to get back the structured output. There may be more than one way to do this (e.g., function calling vs JSON mode) - you can configure which method to use by passing into that method.

Example with OpenAI (supports Tool/function Calling):
```python
from langchain_core.pydantic_v1 import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke"

from langchain_openai import ChatOpenAI

# By default, we will use function_calling:
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = model.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")
# Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!')

# JSON mode:
structured_llm = model.with_structured_output(Joke, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)
# Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!')
```

Example with Mistral (only support function calling):

```python
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")
structured_llm = model.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")
# Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!')
```

Example with Groq (supports Tool/function Calling):

```python
model = ChatGroq()
structured_llm = model.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")
# Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!')

# using JSON mode:
structured_llm = model.with_structured_output(Joke, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)
# Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!')

```

### Chat Models and Caching

LangChain provides an optional caching layer for chat models. It can save you money by reducing the number of API calls you make to the LLM provider, if you're often requesting the same completion multiple times. 

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

from langchain.globals import set_llm_cache

from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

# The second time it is, so it goes faster
llm.predict("Tell me a joke")
```

### Custom Chat Model

Wrapping your LLM with the standard BaseChatModel interface allow you to use your LLM in existing LangChain programs with minimal code modifications. As an bonus, your LLM will automatically become a LangChain Runnable and will benefit from some optimizations out of the box (e.g., batch via a threadpool), async support, the astream_events API, etc.

Chat models take messages as inputs and return a message as output. LangChain has a few built-in message types:
- SystemMessage: 
    - Used for priming AI behavior, usually passed in as the first of a sequence of input messages.
- HumanMessage: 
    - Represents a message from a person interacting with the chat model.
- AIMessage: 
    - Represents a message from the chat model. This can be either text or a request to invoke a tool.
- FunctionMessage / ToolMessage: 
    - Message for passing the results of tool invocation back to the model.
    - ToolMessage and FunctionMessage closely follow OpenAIs function and tool roles.
- AIMessageChunk / HumanMessageChunk: 
    - Chunk variant of each type of message.

Import Example:

```python
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)

# All the chat messages have a streaming variant that contains Chunk in the name.
from langchain_core.messages import (
    AIMessageChunk,
    FunctionMessageChunk,
    HumanMessageChunk,
    SystemMessageChunk,
    ToolMessageChunk,
)
```

Note the streaming variant chunks are used when streaming output from chat models, and they all define an additive property:
```python
AIMessageChunk(content="Hello") + AIMessageChunk(content=" World!")
```

When we inherit from BaseChatModel, we need to implement the following:
- _generate: 
    - Use to generate a chat result from a prompt
    - Required
- _llm_type (property): 
    - Used to uniquely identify the type of the model. Used for logging.
    - Required
- _identifying_params (property): 
    - Represent model parameterization for tracing purposes.
    - Optional
- _stream: 
    - Use to implement streaming.
    - Optional
- _agenerate: 
    - Use to implement a native async method.	
    - Optional
- _astream: 
    - Use to implement async version of _stream.	
    - Optional
    - The _astream implementation uses run_in_executor to launch the sync _stream in a separate thread if _stream is implemented, otherwise it fallsback to use _agenerate.

Example:

```python
from typing import Any, AsyncIterator, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    AsyncCallbackManagerForLLMRun,
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import AIMessageChunk, BaseMessage, HumanMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from langchain_core.runnables import run_in_executor

class CustomChatModelAdvanced(BaseChatModel):
    """A custom chat model that echoes the first `n` characters of the input.

    When contributing an implementation to LangChain, carefully document
    the model including the initialization parameters, include
    an example of how to initialize the model and include any relevant
    links to the underlying models documentation or API.

    Example:

        .. code-block:: python

            model = CustomChatModel(n=2)
            result = model.invoke([HumanMessage(content="hello")])
            result = model.batch([[HumanMessage(content="hello")],
                                 [HumanMessage(content="world")]])
    """

    model_name: str
    """The name of the model"""
    n: int
    """The number of characters from the last message of the prompt to be echoed."""

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Override the _generate method to implement the chat model logic.

        This can be a call to an API, a call to a local model, or any other
        implementation that generates a response to the input prompt.

        Args:
            messages: the prompt composed of a list of messages.
            stop: a list of strings on which the model should stop generating.
                  If generation stops due to a stop token, the stop token itself
                  SHOULD BE INCLUDED as part of the output. This is not enforced
                  across models right now, but it's a good practice to follow since
                  it makes it much easier to parse the output of the model
                  downstream and understand why generation stopped.
            run_manager: A run manager with callbacks for the LLM.
        """
        # Replace this with actual logic to generate a response from a list
        # of messages.
        last_message = messages[-1]
        tokens = last_message.content[: self.n]
        message = AIMessage(
            content=tokens,
            additional_kwargs={},  # Used to add additional payload (e.g., function calling request)
            response_metadata={  # Use for response metadata
                "time_in_seconds": 3,
            },
        )
        ##

        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        """Stream the output of the model.

        This method should be implemented if the model can generate output
        in a streaming fashion. If the model does not support streaming,
        do not implement it. In that case streaming requests will be automatically
        handled by the _generate method.

        Args:
            messages: the prompt composed of a list of messages.
            stop: a list of strings on which the model should stop generating.
                  If generation stops due to a stop token, the stop token itself
                  SHOULD BE INCLUDED as part of the output. This is not enforced
                  across models right now, but it's a good practice to follow since
                  it makes it much easier to parse the output of the model
                  downstream and understand why generation stopped.
            run_manager: A run manager with callbacks for the LLM.
        """
        last_message = messages[-1]
        tokens = last_message.content[: self.n]

        for token in tokens:
            chunk = ChatGenerationChunk(message=AIMessageChunk(content=token))

            if run_manager:
                # This is optional in newer versions of LangChain
                # The on_llm_new_token will be called automatically
                run_manager.on_llm_new_token(token, chunk=chunk)

            yield chunk

        # Let's add some other information (e.g., response metadata)
        chunk = ChatGenerationChunk(
            message=AIMessageChunk(content="", response_metadata={"time_in_sec": 3})
        )
        if run_manager:
            # This is optional in newer versions of LangChain
            # The on_llm_new_token will be called automatically
            run_manager.on_llm_new_token(token, chunk=chunk)
        yield chunk

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "echoing-chat-model-advanced"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.

        This information is used by the LangChain callback system, which
        is used for tracing purposes make it possible to monitor LLMs.
        """
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": self.model_name,
        }
```

The chat model will implement the standard Runnable interface of LangChain:
```python
model = CustomChatModelAdvanced(n=3, model_name="my_custom_model")

model.invoke(
    [
        HumanMessage(content="hello!"),
        AIMessage(content="Hi there human!"),
        HumanMessage(content="Meow!"),
    ]
)
# AIMessage(content='Meo', response_metadata={'time_in_seconds': 3}, id='run-ddb42bd6-4fdd-4bd2-8be5-e11b67d3ac29-0')

model.invoke("hello")
# AIMessage(content='hel', response_metadata={'time_in_seconds': 3}, id='run-4d3cc912-44aa-454b-977b-ca02be06c12e-0')

model.batch(["hello", "goodbye"])
# [AIMessage(content='hel', response_metadata={'time_in_seconds': 3}, id='run-9620e228-1912-4582-8aa1-176813afec49-0'),
#  AIMessage(content='goo', response_metadata={'time_in_seconds': 3}, id='run-1ce8cdf8-6f75-448e-82f7-1bb4a121df93-0')]

for chunk in model.stream("cat"):
    print(chunk.content, end="|")
# c|a|t||

async for chunk in model.astream("cat"):
    print(chunk.content, end="|")
# c|a|t||

# Let's try to use the astream events API which will also help double check that all the callbacks were implemented!
async for event in model.astream_events("cat", version="v1"):
    print(event)
# {'event': 'on_chat_model_start', 'run_id': '125a2a16-b9cd-40de-aa08-8aa9180b07d0', 'name': 'CustomChatModelAdvanced', 'tags': [], 'metadata': {}, 'data': {'input': 'cat'}}
# {'event': 'on_chat_model_stream', 'run_id': '125a2a16-b9cd-40de-aa08-8aa9180b07d0', 'tags': [], 'metadata': {}, 'name': 'CustomChatModelAdvanced', 'data': {'chunk': AIMessageChunk(content='c', id='run-125a2a16-b9cd-40de-aa08-8aa9180b07d0')}}
# {'event': 'on_chat_model_stream', 'run_id': '125a2a16-b9cd-40de-aa08-8aa9180b07d0', 'tags': [], 'metadata': {}, 'name': 'CustomChatModelAdvanced', 'data': {'chunk': AIMessageChunk(content='a', id='run-125a2a16-b9cd-40de-aa08-8aa9180b07d0')}}
# {'event': 'on_chat_model_stream', 'run_id': '125a2a16-b9cd-40de-aa08-8aa9180b07d0', 'tags': [], 'metadata': {}, 'name': 'CustomChatModelAdvanced', 'data': {'chunk': AIMessageChunk(content='t', id='run-125a2a16-b9cd-40de-aa08-8aa9180b07d0')}}
# {'event': 'on_chat_model_stream', 'run_id': '125a2a16-b9cd-40de-aa08-8aa9180b07d0', 'tags': [], 'metadata': {}, 'name': 'CustomChatModelAdvanced', 'data': {'chunk': AIMessageChunk(content='', response_metadata={'time_in_sec': 3}, id='run-125a2a16-b9cd-40de-aa08-8aa9180b07d0')}}
# {'event': 'on_chat_model_end', 'name': 'CustomChatModelAdvanced', 'run_id': '125a2a16-b9cd-40de-aa08-8aa9180b07d0', 'tags': [], 'metadata': {}, 'data': {'output': AIMessageChunk(content='cat', response_metadata={'time_in_sec': 3}, id='run-125a2a16-b9cd-40de-aa08-8aa9180b07d0')}}    
```

### Response metadata

Many model providers include some metadata in their chat generation responses. This metadata can be accessed via the AIMessage.response_metadata: Dict attribute. Depending on the model provider and model configuration, this can contain information like token counts, logprobs, and more.

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata
# {'token_usage': {'completion_tokens': 164,
#   'prompt_tokens': 17,
#   'total_tokens': 181},
#  'model_name': 'gpt-4-turbo',
#  'system_fingerprint': 'fp_76f018034d',
#  'finish_reason': 'stop',
#  'logprobs': None}
```

### LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not serve its own LLMs, but rather provides a standard interface for interacting with many different LLMs. To be specific, this interface is one that takes as input a string and returns a string.

There are lots of LLM providers (OpenAI, Cohere, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

LLMs implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls. LLMs accept strings as inputs, or objects which can be coerced to string prompts, including List[BaseMessage] and PromptValue.

```python
llm.invoke(
    "What are some theories about the relationship between unemployment and inflation?"
)
# '\n\n1. The Phillips Curve Theory: This suggests that there is an inverse relationship between unemployment and inflation, meaning that when unemployment is low, inflation will be higher, and when unemployment is high, inflation will be lower.\n\n2. The Monetarist Theory: This theory suggests that the relationship between unemployment and inflation is weak, and that changes in the money supply are more important in determining inflation.\n\n3. The Resource Utilization Theory: This suggests that when unemployment is low, firms are able to raise wages and prices in order to take advantage of the increased demand for their products and services. This leads to higher inflation.'

for chunk in llm.stream(
    "What are some theories about the relationship between unemployment and inflation?"
):
    print(chunk, end="", flush=True)

llm.batch(
    [
        "What are some theories about the relationship between unemployment and inflation?"
    ]
)

await llm.ainvoke(
    "What are some theories about the relationship between unemployment and inflation?"
)

async for chunk in llm.astream(
    "What are some theories about the relationship between unemployment and inflation?"
):
    print(chunk, end="", flush=True)

await llm.abatch(
    [
        "What are some theories about the relationship between unemployment and inflation?"
    ]
)

async for chunk in llm.astream_log(
    "What are some theories about the relationship between unemployment and inflation?"
):
    print(chunk)
```

### LLMs and LangSmith

All LLMs come with built-in LangSmith tracing. Just set the following environment variables:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY=<your-api-key>
```

and any LLM invocation (whether it's nested in a chain or not) will automatically be traced. A trace will include inputs, outputs, latency, token usage, invocation params, environment params, and more.

### Custom LLM

Wrapping your LLM with the standard LLM interface allow you to use your LLM in existing LangChain programs with minimal code modifications. As an bonus, your LLM will automatically become a LangChain Runnable and will benefit from some optimizations out of the box, async support, the astream_events API, etc.

There are only two required things that a custom LLM needs to implement:
- _call: Takes in a string and some optional stop words, and returns a string. Used by invoke.
- _llm_type: A property that returns a string, used for logging purposes only.

Optional implementations:
- _identifying_params: Used to help with identifying the model and printing the LLM; should return a dictionary. This is a @property.
- _acall: Provides an async native implementation of _call, used by ainvoke.
- _stream: Method to stream the output token by token.
- _astream: Provides an async native implementation of _stream; in newer LangChain versions, defaults to _stream.

Let's implement a simple custom LLM that just returns the first n characters of the input.

```python
from typing import Any, Dict, Iterator, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk


class CustomLLM(LLM):
    """A custom chat model that echoes the first `n` characters of the input.

    When contributing an implementation to LangChain, carefully document
    the model including the initialization parameters, include
    an example of how to initialize the model and include any relevant
    links to the underlying models documentation or API.

    Example:

        .. code-block:: python

            model = CustomChatModel(n=2)
            result = model.invoke([HumanMessage(content="hello")])
            result = model.batch([[HumanMessage(content="hello")],
                                 [HumanMessage(content="world")]])
    """

    n: int
    """The number of characters from the last message of the prompt to be echoed."""

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Run the LLM on the given input.

        Override this method to implement the LLM logic.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of the stop substrings.
                If stop tokens are not supported consider raising NotImplementedError.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            The model output as a string. Actual completions SHOULD NOT include the prompt.
        """
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[: self.n]

    def _stream(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """Stream the LLM on the given prompt.

        This method should be overridden by subclasses that support streaming.

        If not implemented, the default behavior of calls to stream will be to
        fallback to the non-streaming version of the model and return
        the output as a single chunk.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of these substrings.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            An iterator of GenerationChunks.
        """
        for char in prompt[: self.n]:
            chunk = GenerationChunk(text=char)
            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)

            yield chunk

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"
```

This LLM will implement the standard Runnable interface of LangChain:

```python
llm = CustomLLM(n=5)
print(llm)
# Params: {'model_name': 'CustomChatModel'}

llm.invoke("This is a foobar thing")
# 'This '

await llm.ainvoke("world")
# 'world'

llm.batch(["woof woof woof", "meow meow meow"])
# ['woof ', 'meow ']

await llm.abatch(["woof woof woof", "meow meow meow"])
# ['woof ', 'meow ']

async for token in llm.astream("hello"):
    print(token, end="|", flush=True)
# h|e|l|l|o|
```

Let's confirm that in integrates nicely with other LangChain APIs:

```python
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "you are a bot"), ("human", "{input}")]
)

llm = CustomLLM(n=7)
chain = prompt | llm

idx = 0
async for event in chain.astream_events({"input": "hello there!"}, version="v1"):
    print(event)
    idx += 1
    if idx > 7:
        # Truncate
        break
# {'event': 'on_chain_start', 'run_id': '05f24b4f-7ea3-4fb6-8417-3aa21633462f', 'name': 'RunnableSequence', 'tags': [], 'metadata': {}, 'data': {'input': {'input': 'hello there!'}}}
# {'event': 'on_prompt_start', 'name': 'ChatPromptTemplate', 'run_id': '7e996251-a926-4344-809e-c425a9846d21', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'input': {'input': 'hello there!'}}}
# {'event': 'on_prompt_end', 'name': 'ChatPromptTemplate', 'run_id': '7e996251-a926-4344-809e-c425a9846d21', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'input': {'input': 'hello there!'}, 'output': ChatPromptValue(messages=[SystemMessage(content='you are a bot'), HumanMessage(content='hello there!')])}}
# {'event': 'on_llm_start', 'name': 'CustomLLM', 'run_id': 'a8766beb-10f4-41de-8750-3ea7cf0ca7e2', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'input': {'prompts': ['System: you are a bot\nHuman: hello there!']}}}
# {'event': 'on_llm_stream', 'name': 'CustomLLM', 'run_id': 'a8766beb-10f4-41de-8750-3ea7cf0ca7e2', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': 'S'}}
# {'event': 'on_chain_stream', 'run_id': '05f24b4f-7ea3-4fb6-8417-3aa21633462f', 'tags': [], 'metadata': {}, 'name': 'RunnableSequence', 'data': {'chunk': 'S'}}
# {'event': 'on_llm_stream', 'name': 'CustomLLM', 'run_id': 'a8766beb-10f4-41de-8750-3ea7cf0ca7e2', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': 'y'}}
# {'event': 'on_chain_stream', 'run_id': '05f24b4f-7ea3-4fb6-8417-3aa21633462f', 'tags': [], 'metadata': {}, 'name': 'RunnableSequence', 'data': {'chunk': 'y'}}
```

### LLM and Caching

LangChain provides an optional caching layer for LLMs. It can save you money by reducing the number of API calls you make to the LLM provider, if you're often requesting the same completion multiple times.

```python
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

# To make the caching really obvious, lets use a slower model.
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", n=2, best_of=2)

from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

# The second time it is, so it goes faster
llm.predict("Tell me a joke")
```

### LLM and Streaming

All LLMs implement the Runnable interface, which comes with default implementations of all methods, ie. ainvoke, batch, abatch, stream, astream. This gives all LLMs basic support for streaming.

Streaming support defaults to returning an Iterator (or AsyncIterator in the case of async streaming) of a single value, the final result returned by the underlying LLM provider. This obviously doesn't give you token-by-token streaming, which requires native support from the LLM provider, but ensures your code that expects an iterator of tokens can work for any of our LLM integrations.

```python
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, max_tokens=512)
for chunk in llm.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)
```

### Output Parsers

Output parsers are responsible for taking the output of an LLM and transforming it to a more suitable format. This is very useful when you are using LLMs to generate any form of structured data. Besides having a large collection of different types of output parsers, one distinguishing benefit of LangChain OutputParsers is that many of them support streaming.

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:
- "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
- "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.
And then one optional one:
- "Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

Below we go over the main type of output parser, the PydanticOutputParser.

```python
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)
# Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')
```

### Output Parsers and LCEL

Output parsers implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

Output parsers accept a string or BaseMessage as input and can return an arbitrary type.

```python
parser.invoke(output)
```

Instead of manually invoking the parser, we also could've just added it to our Runnable sequence:

```python
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})
# Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!') 
```

While all parsers support the streaming interface, only certain parsers can stream through partially parsed objects, since this is highly dependent on the output type. Parsers which cannot construct partial objects will simply yield the fully parsed output.

The SimpleJsonOutputParser for example can stream through partial outputs:

```python
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

list(json_chain.stream({"question": "Who invented the microscope?"}))
# [{},
#  {'answer': ''},
#  {'answer': 'Ant'},
#  {'answer': 'Anton'},
#  {'answer': 'Antonie'},
#  {'answer': 'Antonie van'},
#  {'answer': 'Antonie van Lee'},
#  {'answer': 'Antonie van Leeu'},
#  {'answer': 'Antonie van Leeuwen'},
#  {'answer': 'Antonie van Leeuwenho'},
#  {'answer': 'Antonie van Leeuwenhoek'}]
```

While the PydanticOutputParser cannot:

```python
list(chain.stream({"query": "Tell me a joke."}))
# [Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')]
```

### Custom Output Parsers

In some situations you may want to implement a custom parser to structure the model output into a custom format.

There are two ways to implement a custom parser:
- Using RunnableLambda (non-streaming) or RunnableGenerator (streaming) in LCEL -- we strongly recommend this for most use cases
- By inherting from one of the base classes for out parsing -- this is the hard way of doing things

Example using Runnable Lambdas and Generators:

Here, we will make a simple parser that inverts the case of the output from the model. For example, if the model outputs: "Meow", the parser will produce "mEOW".

```python
from typing import Iterable

from langchain_anthropic.chat_models import ChatAnthropic
from langchain_core.messages import AIMessage, AIMessageChunk

model = ChatAnthropic(model_name="claude-2.1")


def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = model | parse
chain.invoke("hello")
# 'hELLO!'
```

LCEL automatically upgrades the function parse to RunnableLambda(parse) when composed using a | syntax. If you don't like that you can manually import RunnableLambda and then run: parse = RunnableLambda(parse).

Streaming doesn't work in the original implementation because the parser (parse) processes the entire response at once, aggregating all chunks before applying the transformation. In a streaming scenario, the output is expected to be processed and returned incrementally as each chunk is received. However, with the original parse, the response from the model is first fully collected, and only after that is the case inversion applied. This behavior inherently defeats the purpose of streaming. Failure to work:

```python
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)
# i'M cLAUDE, AN ai ASSISTANT CREATED BY aNTHROPIC TO BE HELPFUL, HARMLESS, AND HONEST.|
```

In a streaming scenario, the output needs to be generated chunk by chunk as the input becomes available. When using RunnableLambda (automatically applied by LangChain when | is used), the parse function is wrapped in a non-streaming runnable. The wrapping assumes the function processes a complete input and produces a single output, which is incompatible with streaming.

To enable streaming, the parser must:
- Accept a stream of chunks (e.g., Iterable[AIMessageChunk]) instead of a complete AIMessage.
- Yield processed chunks incrementally as they are received.

The corrected implementation uses RunnableGenerator, which is designed for streaming tasks. This allows the parser to process and yield each chunk of the response as it arrives.

Example:

```python
from typing import Iterable
from langchain_core.runnables import RunnableGenerator
from langchain_anthropic.chat_models import ChatAnthropic
from langchain_core.messages import AIMessageChunk

# Define the model
model = ChatAnthropic(model_name="claude-2.1")

# Define the streaming parser
def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        # Process each chunk and yield the result incrementally
        yield chunk.content.swapcase()

# Wrap the parser in a RunnableGenerator for streaming
streaming_parse = RunnableGenerator(streaming_parse)

# Combine the model and the streaming parser
chain = model | streaming_parse

# Use the chain in a streaming scenario
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)
```

Another approach to implement a parser is by inherting from BaseOutputParser, BaseGenerationOutputParser or another one of the base parsers depending on what you need to do. In general, we do not recommend this approach for most use cases as it results in more code to write without significant benefits.

### Output Parsers: CSV parser

This output parser can be used when you want to return a list of comma-separated items.

```python
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

chain.invoke({"subject": "ice cream flavors"})
# ['Vanilla',
#  'Chocolate',
#  'Strawberry',
#  'Mint Chocolate Chip',
#  'Cookies and Cream']

for s in chain.stream({"subject": "ice cream flavors"}):
    print(s)
```

### Output Parsers: Datetime parser

This OutputParser can be used to parse LLM output into datetime format.

```python
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | OpenAI() | output_parser

output = chain.invoke({"question": "when was bitcoin founded?"})

print(output)
# 2009-01-03 18:15:05
```


### Output Parsers: Enum parser

This notebook shows how to use an Enum output parser.

```python
from langchain.output_parsers.enum import EnumOutputParser

from enum import Enum

class Colors(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

parser = EnumOutputParser(enum=Colors)

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """What color eyes does this person have?

> Person: {person}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())
chain = prompt | ChatOpenAI() | parser

chain.invoke({"person": "Frank Sinatra"})
# <Colors.BLUE: 'blue'>
```

### Output Parsers: JSON parser

This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are leaky abstractions! You'll have to use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, DaVinci can do reliably but Curie's ability already drops off dramatically.

You can optionally use Pydantic to declare your data model.

```python
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})
# {'setup': "Why don't scientists trust atoms?",
#  'punchline': 'Because they make up everything!'}
```

This output parser supports streaming.

for s in chain.stream({"query": joke_query}):
    print(s)
<!-- {'setup': ''}
{'setup': 'Why'}
{'setup': 'Why don'}
{'setup': "Why don't"}
{'setup': "Why don't scientists"}
{'setup': "Why don't scientists trust"}
{'setup': "Why don't scientists trust atoms"}
{'setup': "Why don't scientists trust atoms?", 'punchline': ''}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'} -->

You can also use this without Pydantic. This will prompt it return JSON, but doesn't provide specific about what the schema should be.

```python
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})
# {'joke': "Why don't scientists trust atoms? Because they make up everything!"}
```

### Output Parsers: OpenAI Functions

These output parsers use OpenAI function calling to structure its outputs. This means they are only usable with models that support function calling. There are a few different variants:

- JsonOutputFunctionsParser: Returns the arguments of the function call as JSON
- PydanticOutputFunctionsParser: Returns the arguments of the function call as a Pydantic Model
- JsonKeyOutputFunctionsParser: Returns the value of specific key in the function call as JSON
- PydanticAttrOutputFunctionsParser: Returns the value of specific key in the function call as a Pydantic Model

```python
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


openai_functions = [convert_pydantic_to_openai_function(Joke)]

model = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

parser = JsonOutputFunctionsParser()

chain = prompt | model.bind(functions=openai_functions) | parser

chain.invoke({"input": "tell me a joke"})
# {'setup': "Why don't scientists trust atoms?",
#  'punchline': 'Because they make up everything!'}

for s in chain.stream({"input": "tell me a joke"}):
    print(s)
# {}
# {'setup': ''}
# {'setup': 'Why'}
# {'setup': 'Why don'}
# {'setup': "Why don't"}
# {'setup': "Why don't scientists"}
# {'setup': "Why don't scientists trust"}
# {'setup': "Why don't scientists trust atoms"}
# {'setup': "Why don't scientists trust atoms?", 'punchline': ''}
# {'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}
# {'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}
# {'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}
# {'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}
# {'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}
# {'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}

PydanticOutputFunctionsParser builds on top of JsonOutputFunctionsParser but passes the results to a Pydantic Model. This allows for further validation should you choose. 

```python
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticOutputFunctionsParser(pydantic_schema=Joke)

openai_functions = [convert_pydantic_to_openai_function(Joke)]
chain = prompt | model.bind(functions=openai_functions) | parser

chain.invoke({"input": "tell me a joke"})
# Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')
```

Notice the return value of above is a pydantic object!

### Output Parsers: OpenAI Tools

These output parsers extract tool calls from OpenAI's function calling API responses. This means they are only usable with models that support function calling, and specifically the latest tools and tool_choice parameters. 

There are a few different variants of output parsers:
- JsonOutputToolsParser: Returns the arguments of the function call as JSON
- JsonOutputKeyToolsParser: Returns the value of specific key in the function call as JSON
- PydanticToolsParser: Returns the arguments of the function call as a Pydantic Model

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])

model.kwargs["tools"]
# [{'type': 'function',
#   'function': {'name': 'Joke',
#    'description': 'Joke to tell user.',
#    'parameters': {'type': 'object',
#     'properties': {'setup': {'description': 'question to set up a joke',
#       'type': 'string'},
#      'punchline': {'description': 'answer to resolve the joke',
#       'type': 'string'}},
#     'required': ['setup', 'punchline']}}}]

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

from langchain.output_parsers.openai_tools import JsonOutputToolsParser

parser = JsonOutputToolsParser()

chain = prompt | model | parser

chain.invoke({"input": "tell me a joke"})
# [{'type': 'Joke',
#   'args': {'setup': "Why don't scientists trust atoms?",
#    'punchline': 'Because they make up everything!'}}]
```

To include the tool call id we can specify return_id=True:

```python
parser = JsonOutputToolsParser(return_id=True)
chain = prompt | model | parser
chain.invoke({"input": "tell me a joke"})
# [{'type': 'Joke',
#   'args': {'setup': "Why don't scientists trust atoms?",
#    'punchline': 'Because they make up everything!'},
#   'id': 'call_Isuoh0RTeQzzOKGg5QlQ7UqI'}]
```

PydanticToolsParser builds on top of JsonOutputToolsParser but passes the results to a Pydantic Model. This allows for further validation should you choose.

```python
from langchain.output_parsers.openai_tools import PydanticToolsParser

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticToolsParser(tools=[Joke])

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])
chain = prompt | model | parser

chain.invoke({"input": "tell me a joke"})
# [Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')]
```

### Output Parsers: Output-fixing parser

This output parser wraps another output parser, and in the event that the first one fails it calls out to another LLM to fix any errors. But we can do other things besides throw errors. Specifically, we can pass the misformatted output, along with the formatted instructions, to the model and ask it to fix it. For this example, we'll use the above Pydantic output parser. Here's what happens if we pass it a result that does not comply with the schema:

```python
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

parser.parse(misformatted)
# ---------------------------------------------------------------------------
# ``````output
# JSONDecodeError                           Traceback (most recent call last)
# ``````output
# File ~/workplace/langchain/libs/langchain/langchain/output_parsers/pydantic.py:29, in PydanticOutputParser.parse(self, text)
#      28     json_str = match.group()
# ---> 29 json_object = json.loads(json_str, strict=False)
#      30 return self.pydantic_object.parse_obj(json_object)
# ``````output
# File ~/.pyenv/versions/3.10.1/lib/python3.10/json/__init__.py:359, in loads(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)
#     358     kw['parse_constant'] = parse_constant
# --> 359 return cls(**kw).decode(s)
# ``````output
# File ~/.pyenv/versions/3.10.1/lib/python3.10/json/decoder.py:337, in JSONDecoder.decode(self, s, _w)
#     333 """Return the Python representation of ``s`` (a ``str`` instance
#     334 containing a JSON document).
#     335 
#     336 """
# --> 337 obj, end = self.raw_decode(s, idx=_w(s, 0).end())
#     338 end = _w(s, end).end()
# ``````output
# File ~/.pyenv/versions/3.10.1/lib/python3.10/json/decoder.py:353, in JSONDecoder.raw_decode(self, s, idx)
#     352 try:
# --> 353     obj, end = self.scan_once(s, idx)
#     354 except StopIteration as err:
# ``````output
# JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
# ``````output

# During handling of the above exception, another exception occurred:
# ``````output
# OutputParserException                     Traceback (most recent call last)
# ``````output
# Cell In[4], line 1
# ----> 1 parser.parse(misformatted)
# ``````output
# File ~/workplace/langchain/libs/langchain/langchain/output_parsers/pydantic.py:35, in PydanticOutputParser.parse(self, text)
#      33 name = self.pydantic_object.__name__
#      34 msg = f"Failed to parse {name} from completion {text}. Got: {e}"
# ---> 35 raise OutputParserException(msg, llm_output=text)
# ``````output
# OutputParserException: Failed to parse Actor from completion {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}. Got: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
```

Now we can construct and use a OutputFixingParser. This output parser takes as an argument another output parser but also an LLM with which to try to correct any formatting mistakes.

```python
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

new_parser.parse(misformatted)
# Actor(name='Tom Hanks', film_names=['Forrest Gump'])
```


### Output Parsers: Pandas DataFrame Parser

The PandasDataFrameOutputParser in LangChain:
- Allows you to query a DataFrame using an LLM.
- Structures the LLM's output to match the format of a Pandas DataFrame operation.
    - The parser acts as a bridge between the natural language output generated by the language model (LLM) and the structured, programmatic format required to work with a Pandas DataFrame.
    - The parser provides the model with clear instructions on how to format its output.
        - Example: A column retrieval query should produce a dictionary with column names as keys and values as data.
        - Example: A row retrieval query should produce a dictionary with row indices as keys and column-value mappings as values.
    - The parser validates Against the DataFrame Schema:
        - The parser ensures that the response only references valid columns or operations defined in the DataFrame.
        - Example: A query asking for a non-existent column will fail. 
    - The parser transforms the LLM's Output:
         - Converts the natural language output into a structured, machine-readable format (like a dictionary or JSON) that aligns with the DataFrame's data and structure.

In the example below, the parser provides format instructions for the LLM to interpret user queries. Validates and processes the response to ensure it matches the expected structure. Facilitates operations on a Pandas DataFrame, such as retrieving rows, columns, or performing calculations.

```python
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    {
        "num_legs": [2, 4, 8, 0],
        "num_wings": [2, 0, 0, 0],
        "num_specimen_seen": [10, 2, 1, 8],
    }
)

# Set up a parser + inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)

# Here's an example of a column operation being performed.
df_query = "Retrieve the num_wings column."

# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)
# {'num_wings': {0: 2,
#                1: 0,
#                2: 0,
#                3: 0}}

# Here's an example of a row operation being performed.
df_query = "Retrieve the first row."

# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)
# {'0': {'num_legs': 2,
#        'num_specimen_seen': 10,
#        'num_wings': 2}}

# Here's an example of a random Pandas DataFrame operation limiting the number of rows
df_query = "Retrieve the average of the num_legs column from rows 1 to 3."

# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

print(parser_output)
# {'mean': 4.0}

# Here's an example of a poorly formatted query
df_query = "Retrieve the mean of the num_fingers column."

# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})
# OutputParserException: Invalid column: num_fingers. Please check the format instructions.
```

Why the Last Query Fails:
- Invalid Column:
    - The DataFrame does not contain a column named num_fingers.
    - The parser validates the query and finds that it references a non-existent column.
- Validation Failure:
    - The PandasDataFrameOutputParser checks the LLM's response against the DataFrame's structure.
    - If the response contains an invalid column, the parser raises an OutputParserException.
- Error Raised:
    - OutputParserException: Invalid column: num_fingers. Please check the format instructions.

The Pandas DataFrame Parser does not replace the Pandas DataFrame Agent! Pandas DataFrame Agent provides a more general-purpose, conversational interface for performing complex and multi-step operations. It can can handle multi-step reasoning, chain operations, and respond dynamically to arbitrary queries. It is best for complex, multi-step operations requiring reasoning and decision-making (e.g., combining multiple columns, filtering rows dynamically). It acts as the decision-maker, dynamically figuring out what DataFrame operations to perform and executing Python code.

### Output Parsers: Pydantic parser

This output parser allows users to specify an arbitrary Pydantic Model and query LLMs for outputs that conform to that schema. Keep in mind that large language models are leaky abstractions! You'll have to use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, DaVinci can do reliably but Curie's ability already drops off dramatically.

Use Pydantic to declare your data model. Pydantic's BaseModel is like a Python dataclass, but with actual type checking + coercion.

```python
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})
# Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

# Here's another example, but with a compound typed field (e.g. List[str]).
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": actor_query})
# Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Cast Away', 'Saving Private Ryan', 'Toy Story', 'The Green Mile'])
```

### Output Parsers: Retry Parser

While in some cases it is possible to fix any parsing mistakes by only looking at the output, in other cases it isn't. An example of this is when the output is not just in the incorrect format, but is partially complete. Consider the below example.

```python
from langchain.output_parsers import (
    OutputFixingParser,
    PydanticOutputParser,
)
from langchain_core.prompts import (
    PromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAI

template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""


class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

bad_response = '{"action": "search"}'

parser.parse(bad_response)
# OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
# action_input
#   field required (type=value_error.missing)
```

If we try to use the OutputFixingParser to fix this error, it will be confused - namely, it doesn't know what to actually put for action input.

```python
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

fix_parser.parse(bad_response)
# Action(action='search', action_input='input')
```

Instead, we can use the RetryOutputParser, which passes in the prompt (as well as the original output) to try again to get a better response.

```python
from langchain.output_parsers import RetryOutputParser

retry_parser = RetryOutputParser.from_llm(parser=parser, llm=OpenAI(temperature=0))

retry_parser.parse_with_prompt(bad_response, prompt_value)
# Action(action='search', action_input='leo di caprio girlfriend')
```

We can also add the RetryOutputParser easily with a custom chain which transform the raw LLM/ChatModel output into a more workable format.

```python
from langchain_core.runnables import RunnableLambda, RunnableParallel

completion_chain = prompt | OpenAI(temperature=0)

main_chain = RunnableParallel(
    completion=completion_chain, prompt_value=prompt
) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))


main_chain.invoke({"query": "who is leo di caprios gf?"})
# Action(action='search', action_input='leo di caprio girlfriend')
```
- A user query: "Who is Leo DiCaprio's girlfriend?"
- The prompt instructs the LLM to return an object with the fields:
    - action: The action to take.
    - action_input: The input for the action.
- The initial LLM output: {"action": "search"}. This fails validation because the action_input field is missing. The action_input field is required by the schema defined in the Action class.
- The RetryOutputParser:
    - Takes the initial invalid output.
    - Reuses the original prompt (prompt_value) to ask the LLM to generate a better response.
    - Provides the invalid output as additional context to guide the LLM.
- The LLM now generates a corrected response: {"action": "search", "action_input": "leo di caprio girlfriend"}

It's important to note that the output of RunnableParallel is piped to RunnableLambda which does the retry work. The purpose of RunnableParallel here is to parallelize independent tasks. In this example: The retry itself is sequential and dependent on the raw output, so it cannot and should not run in parallel.

The retry loop in the RetryOutputParser is encapsulated within its parse_with_prompt method. In the context of RunnableParallel, the retry loop doesn't run in parallel itself—it runs sequentially within the RetryOutputParser as part of its task.

Inside RetryOutputParser.parse_with_prompt:
- Initial Attempt: The parser tries to validate the raw output (bad_response) using parser.parse().
- Error Handling: If validation fails (e.g., due to missing fields), it catches the error and retries with the LLM.
- Re-Prompting Logic: The original prompt (prompt_value) is sent to the LLM again. The invalid output is included as additional context, helping the LLM generate a better response.
- Retry Loop: This retry process continues until the output passes validation or a maximum retry count is reached.

### Retrieval

Many LLM applications require user-specific data that is not part of the model's training set. The primary way of accomplishing this is through Retrieval Augmented Generation (RAG). In this process, external data is retrieved and then passed to the LLM when doing the generation step.

1) Document loaders

Document loaders load documents from many different sources. LangChain provides over 100 different document loaders as well as integrations with other major providers in the space, like AirByte and Unstructured. LangChain provides integrations to load all types of documents (HTML, PDF, code) from all types of locations (private S3 buckets, public websites).

2) Text Splitting

A key part of retrieval is fetching only the relevant parts of documents. This involves several transformation steps to prepare the documents for retrieval. One of the primary ones here is splitting (or chunking) a large document into smaller chunks. LangChain provides several transformation algorithms for doing this, as well as logic optimized for specific document types (code, markdown, etc).

3) Text embedding models

Another key part of retrieval is creating embeddings for documents. Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of a text that are similar. LangChain provides integrations with over 25 different embedding providers and methods, from open-source to proprietary API, allowing you to choose the one best suited for your needs. LangChain provides a standard interface, allowing you to easily swap between models.

4) Vector stores

With the rise of embeddings, there has emerged a need for databases to support efficient storage and searching of these embeddings. LangChain provides integrations with over 50 different vectorstores, from open-source local ones to cloud-hosted proprietary ones, allowing you to choose the one best suited for your needs. LangChain exposes a standard interface, allowing you to easily swap between vector stores.

5) Retrievers

Once the data is in the database, you still need to retrieve it. LangChain supports many different retrieval algorithms and is one of the places where we add the most value. LangChain supports basic methods that are easy to get started - namely simple semantic search. However, we have also added a collection of algorithms on top of this to increase performance. These include:
- Parent Document Retriever: This allows you to create multiple embeddings per parent document, allowing you to look up smaller chunks but return larger context.
- Self Query Retriever: User questions often contain a reference to something that isn't just semantic but rather expresses some logic that can best be represented as a metadata filter. Self-query allows you to parse out the semantic part of a query from other metadata filters present in the query.
- Ensemble Retriever: Sometimes you may want to retrieve documents from multiple different sources, or using multiple different algorithms. The ensemble retriever allows you to easily do this.

6) Indexing

The LangChain Indexing API syncs your data from any source into a vector store, helping you:
- Avoid writing duplicated content into the vector store
- Avoid re-writing unchanged content
- Avoid re-computing embeddings over unchanged content


### Retrieval: Document Loaders

Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

Document loaders provide a "load" method for loading data as documents from a configured source. They optionally implement a "lazy load" as well for lazily loading data into memory.

### Retrieval: Custom Document Loader

Applications based on LLMs frequently entail extracting data from databases or files, like PDFs, and converting it into a format that LLMs can utilize. In LangChain, this usually involves creating Document objects, which encapsulate the extracted text (page_content) along with metadata—a dictionary containing details about the document, such as the author's name or the date of publication.

Document objects are often formatted into prompts that are fed into an LLM, allowing the LLM to use the information in the Document to generate a desired response (e.g., summarizing the document). Documents can be either used immediately or indexed into a vectorstore for future retrieval and use.

The main abstractions for Document Loading are:
- Document: Contains text and metadata
- BaseLoader: Use to convert raw data into Documents
- Blob: A representation of binary data that's located either in a file or in memory
- BaseBlobParser: Logic to parse a Blob to yield Document objects

This guide will demonstrate how to write custom document loading:
- Create a standard document Loader by sub-classing from BaseLoader.
- Create a parser using BaseBlobParser and use it in conjunction with Blob and BlobLoaders. This is useful primarily when working with files.

A document loader can be implemented by sub-classing from a BaseLoader which provides a standard interface for loading documents:
- lazy_load: Used to load documents one by one lazily. Use for production code.
- alazy_load: Async variant of lazy_load
- load: Used to load all the documents into memory eagerly. Use for prototyping or interactive work.
- aload: Used to load all the documents into memory eagerly. Use for prototyping or interactive work. Added in 2024-04 to LangChain.

All configuration is expected to be passed through the initializer (init). This was a design choice made by LangChain to make sure that once a document loader has been instantiated it has all the information needed to load documents.

Let's create an example of a standard document loader that loads a file and creates a document from each line in the file:
```python 
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

    # alazy_load is OPTIONAL.
    # If you leave out the implementation, a default implementation which delegates to lazy_load will be used!
    async def alazy_load(
        self,
    ) -> AsyncIterator[Document]:  # <-- Does not take any arguments
        """An async lazy loader that reads a file line by line."""
        # Requires aiofiles
        # Install with `pip install aiofiles`
        # https://github.com/Tinche/aiofiles
        import aiofiles

        async with aiofiles.open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            async for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1
```

To test out the document loader, we need a file with some quality content.

```python
with open("./meow.txt", "w", encoding="utf-8") as f:
    quality_content = "meow meow🐱 \n meow meow🐱 \n meow😻😻"
    f.write(quality_content)

loader = CustomDocumentLoader("./meow.txt")

## Test out the lazy load interface
for doc in loader.lazy_load():
    print()
    print(type(doc))
    print(doc)
# <class 'langchain_core.documents.base.Document'>
# page_content='meow meow🐱 \n' metadata={'line_number': 0, 'source': './meow.txt'}

# <class 'langchain_core.documents.base.Document'>
# page_content=' meow meow🐱 \n' metadata={'line_number': 1, 'source': './meow.txt'}

# <class 'langchain_core.documents.base.Document'>
# page_content=' meow😻😻' metadata={'line_number': 2, 'source': './meow.txt'}

## Test out the async implementation
async for doc in loader.alazy_load():
    print()
    print(type(doc))
    print(doc)
```

load() can be helpful in an interactive environment such as a jupyter notebook. Avoid using it for production code since eager loading assumes that all the content can fit into memory, which is not always the case, especially for enterprise data.

Many document loaders involve parsing files. The difference between such loaders usually stems from how the file is parsed rather than how the file is loaded. For example, you can use open to read the binary content of either a PDF or a markdown file, but you need different parsing logic to convert that binary data into text. As a result, it can be helpful to decouple the parsing logic from the loading logic, which makes it easier to re-use a given parser regardless of how the data was loaded.

### Retrieval: Custom Document Parser

Do not conflate the Loader with the Parser. The BaseLoader is responsible for directly loading documents from a source (e.g., files, APIs, databases) and converting them into Document objects. It works at a higher level of abstraction.
- Handles complete loading and parsing: Combines both fetching the data and parsing it into Document objects.
- Designed for specific sources like text files, APIs, or databases.
- Provides standard methods:
    - lazy_load: Loads documents one at a time (lazy loading).
    - alazy_load: Async variant of lazy loading.
    - load: Loads all documents into memory at once (eager loading).
    - aload: Async variant of eager loading.

Use BaseLoader when the source (e.g., a file, database, or API) is known, and you want to quickly transform raw data into Document objects without extra modularity.

BaseBlobParser

**The BaseBlobParser focuses solely on parsing raw binary data (blobs) into Document objects. It is designed for scenarios where the data-loading logic is decoupled from the data-parsing logic**.
- Works with Blob objects, which represent binary data from either files or memory.
- Only parses the provided binary data into Document objects—it does not handle loading the data.
- Provides methods like:
    - lazy_parse: Parses blobs one by one (lazy parsing).

Use BaseBlobParser when:
- You need modularity: Data loading (e.g., fetching files) and data parsing are handled separately.
- The same parsing logic can be reused across different types of data sources (e.g., files, in-memory data, APIs).
- Suitable for files or binary data that require specialized parsing (e.g., PDFs, CSVs, JSON).

A BaseBlobParser is an interface that accepts a blob and outputs a list of Document objects. A blob is a representation of data that lives either in memory or in a file. LangChain python has a Blob primitive which is inspired by the Blob WebAPI spec.

```python
from langchain_core.document_loaders import BaseBlobParser, Blob

class MyParser(BaseBlobParser):
    """A simple parser that creates a document from each line."""

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Parse a blob into a document line by line."""
        line_number = 0
        with blob.as_bytes_io() as f:
            for line in f:
                line_number += 1
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": blob.source},
                )

blob = Blob.from_path("./meow.txt")
parser = MyParser()

list(parser.lazy_parse(blob))
# [Document(page_content='meow meow🐱 \n', metadata={'line_number': 1, 'source': './meow.txt'}),
#  Document(page_content=' meow meow🐱 \n', metadata={'line_number': 2, 'source': './meow.txt'}),
#  Document(page_content=' meow😻😻', metadata={'line_number': 3, 'source': './meow.txt'})]
```

**Using the blob API also allows one to load content direclty from memory without having to read it from a file!**

```python
blob = Blob(data=b"some data from memory\nmeow")
list(parser.lazy_parse(blob))
# [Document(page_content='some data from memory\n', metadata={'line_number': 1, 'source': None}),
#  Document(page_content='meow', metadata={'line_number': 2, 'source': None})]
```

The following illustrates features of the Blob API:

```python
blob = Blob.from_path("./meow.txt", metadata={"foo": "bar"})

blob.encoding
# 'utf-8'

blob.as_bytes()
# b'meow meow\xf0\x9f\x90\xb1 \n meow meow\xf0\x9f\x90\xb1 \n meow\xf0\x9f\x98\xbb\xf0\x9f\x98\xbb'

blob.as_string()
# 'meow meow🐱 \n meow meow🐱 \n meow😻😻'

blob.as_bytes_io()
# <contextlib._GeneratorContextManager at 0x743f34324450>

blob.metadata
# {'foo': 'bar'}

blob.source
# './meow.txt'
```

Blob Loaders

Let's distinguish DocumentLoader from BlobLoader: A DocumentLoader is a higher-level abstraction that directly converts raw data (e.g., from files, APIs, or databases) into Document objects, which include:
- page_content: The main text content of the document.
- metadata: Additional information about the document (e.g., source, author, creation date).

DocumentLoader abstracts both data retrieval and parsing logic into a single interface. It is ideal for straightforward use cases where parsing is tightly coupled to data loading.

Examples of Use Cases:
- Loading text files where each line corresponds to a document.
- Loading a JSON file where each record represents a document.

A BlobLoader focuses on fetching raw binary data (referred to as "blobs") from a storage location. Unlike a DocumentLoader, it is not responsible for parsing the raw data into documents; instead, it works in tandem with a BlobParser, which handles parsing logic.

Responsibilities of BlobLoader:
- Encapsulates the loading logic for retrieving blobs from a specific source, such as:
    - Local files (e.g., via FileSystemBlobLoader).
    - Cloud storage (future support).
- Provides blobs as raw binary data or streams to be processed further by a parser.
- While a blob parser encapsulates the logic needed to parse binary data into documents, blob loaders encapsulate the logic that's necessary to load blobs from a given storage location. A the moment, LangChain only supports FileSystemBlobLoader.

You can use the FileSystemBlobLoader to load blobs and then use the parser to parse them.

```python
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader

blob_loader = FileSystemBlobLoader(path=".", glob="*.mdx", show_progress=True)

parser = MyParser()
for blob in blob_loader.yield_blobs():
    for doc in parser.lazy_parse(blob):
        print(doc)
        break
# page_content='# Microsoft Office\n' metadata={'line_number': 1, 'source': 'office_file.mdx'}
# page_content='# Markdown\n' metadata={'line_number': 1, 'source': 'markdown.mdx'}
# page_content='# JSON\n' metadata={'line_number': 1, 'source': 'json.mdx'}
# page_content='---\n' metadata={'line_number': 1, 'source': 'pdf.mdx'}
# page_content='---\n' metadata={'line_number': 1, 'source': 'index.mdx'}
# page_content='# File Directory\n' metadata={'line_number': 1, 'source': 'file_directory.mdx'}
# page_content='# CSV\n' metadata={'line_number': 1, 'source': 'csv.mdx'}
# page_content='# HTML\n' metadata={'line_number': 1, 'source': 'html.mdx'}
```

### Retrieval: Generic Loader

LangChain has a GenericLoader abstraction which composes a BlobLoader with a BaseBlobParser. GenericLoader is meant to provide standardized classmethods that make it easy to use existing BlobLoader implementations. At the moment, only the FileSystemBlobLoader is supported.

```python
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader.from_filesystem(
    path=".", glob="*.mdx", show_progress=True, parser=MyParser()
)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")
# page_content='# Microsoft Office\n' metadata={'line_number': 1, 'source': 'office_file.mdx'}
# page_content='\n' metadata={'line_number': 2, 'source': 'office_file.mdx'}
# page_content='>[The Microsoft Office](https://www.office.com/) suite of productivity software includes Microsoft Word, Microsoft Excel, Microsoft PowerPoint, Microsoft Outlook, and Microsoft OneNote. It is available for Microsoft Windows and macOS operating systems. It is also available on Android and iOS.\n' metadata={'line_number': 3, 'source': 'office_file.mdx'}
# page_content='\n' metadata={'line_number': 4, 'source': 'office_file.mdx'}
# page_content='This covers how to load commonly used file formats including `DOCX`, `XLSX` and `PPTX` documents into a document format that we can use downstream.\n' metadata={'line_number': 5, 'source': 'office_file.mdx'}
# ... output truncated for demo purposes
```

### Retrieval: CSV Loader

A comma-separated values (CSV) file is a delimited text file that uses a comma to separate values. Each line of the file is a data record. Each record consists of one or more fields, separated by commas.

Each Row in the CSV Becomes a Separate Document:
- The loader treats each row in the CSV file as an individual Document object.
- This means that:
    - The content of the row (all columns combined) becomes the page_content of the Document.
    - Metadata, such as column names and other details, can also be included in the metadata field of the Document.

CSV File Example:
```csv
name,age,city
Alice,30,New York
Bob,25,Los Angeles
Charlie,35,Chicago
```

If you load this CSV file using the CSV Document Loader, you’ll get three separate Document objects, one for each row:

```python
Document(
    page_content="Alice,30,New York",
    metadata={"source": "data.csv", "line_number": 1}
)

Document(
    page_content="Bob,25,Los Angeles",
    metadata={"source": "data.csv", "line_number": 2}
)

Document(
    page_content="Charlie,35,Chicago",
    metadata={"source": "data.csv", "line_number": 3}
)
```
- You can perform operations like querying, embedding, or indexing on each row separately, which is useful for tasks like retrieval-augmented generation (RAG). 
- Each document can carry metadata specific to its row, such as the row number, file name, or specific column names. 
- Processing each row as a separate document helps with handling large CSVs by working on chunks rather than loading the entire file into memory.

How to include Column Headers for Context

When using the CSV Document Loader, each row of the CSV file is converted into a Document object with:
- page_content: The raw text of the row (comma-separated values).
- metadata: This includes the column headers as keys and the corresponding row values as their values.

CSV File Example:
```csv
name,age,city
Alice,30,New York
Bob,25,Los Angeles
Charlie,35,Chicago
```

You can store the column headers and their respective values in the metadata:

```python
Document(
    page_content="Alice,30,New York",
    metadata={
        "source": "data.csv",
        "line_number": 1,
        "name": "Alice",
        "age": "30",
        "city": "New York"
    }
)

Document(
    page_content="Bob,25,Los Angeles",
    metadata={
        "source": "data.csv",
        "line_number": 2,
        "name": "Bob",
        "age": "25",
        "city": "Los Angeles"
    }
)

Document(
    page_content="Charlie,35,Chicago",
    metadata={
        "source": "data.csv",
        "line_number": 3,
        "name": "Charlie",
        "age": "35",
        "city": "Chicago"
    }
)
```

Associating column names to their respective values in a Retrieval-Augmented Generation (RAG) flow requires careful design, as vector databases typically work with unstructured or semi-structured text rather than tabular data directly. To ensure that both column names (headers) and their corresponding values are meaningfully represented in the vector embeddings, you can flatten the structured metadata into text during vectorization while maintaining the context.
- Flatten the structured data (metadata) into a natural language or key-value text representation.
- This provides the necessary context to the vectorizer, associating column names with their values.

For a CSV row like:

```csv
name,age,city
Alice,30,New York
```

- Flatten the metadata into text: "name: Alice, age: 30, city: New York"
- Natural Language Format: "The person's name is Alice, their age is 30, and they live in New York."
- This combined text becomes the input for vectorization.


The Source Column

The source_column argument in the LangChain CSV Loader allows you to specify a particular column in the CSV file that should be used as the source identifier for each document created from a row. By default, the CSV Loader assigns the same file_path as the source metadata for all documents created from the CSV file. This means all rows (documents) share the same source, which may not be ideal for certain use cases.

If you load a CSV file without specifying source_column:

```python
loader = CSVLoader(file_path='./example_data/mlb_teams_2012.csv')
data = loader.load()
```

Metadata for each Document:
```python
{'source': './example_data/mlb_teams_2012.csv', 'row': 0}
{'source': './example_data/mlb_teams_2012.csv', 'row': 1}
```

If you load the same CSV file but specify source_column="Team":

```python
loader = CSVLoader(file_path='./example_data/mlb_teams_2012.csv', source_column="Team")
data = loader.load()
```

Metadata for each Document:
```python
{'source': 'Nationals', 'row': 0}
{'source': 'Reds', 'row': 1}
{'source': 'Yankees', 'row': 2}
```

Now, the source metadata reflects the value in the "Team" column, which uniquely identifies each row (document).

In retrieval-augmented generation (RAG) workflows, this feature is especially useful when:
- You retrieve relevant rows (documents) from a vector database.
- You want to include the source of the information in the model's output.

Example

Query: "Which MLB team had the highest payroll in 2012?"

If source_column="Team" is used, the retrieved document for "Yankees" will have the source set to "Yankees". This allows the model to output:

```text
The Yankees had the highest payroll in 2012 with $197.96 million.
Source: Yankees
```



### Retrieval: File Directory Loader

We can use the glob parameter to control which files to load. Note that here it doesn't load the .rst file or the .html files.

```python
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('../', glob="**/*.md")
docs = loader.load()
len(docs)
# 1
```

START HERE:
https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/file_directory/
https://python.langchain.com/v0.1/docs/expression_language/

### LCEL and Runnable Interface